In [ ]:
import pandas as pd
from pathlib import Path
from anthropic import Anthropic
import httpx
import random
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Initialize Anthropic client with custom httpx client to handle SSL issues
http_client = httpx.Client(verify=False)
client = Anthropic(
    api_key=os.getenv("ANTHROPIC_API_KEY"),
    http_client=http_client
)

# Load base questions using fastparquet engine
base_questions_file = Path('../data/baseQuestions.parquet')
df_base = pd.read_parquet(base_questions_file, engine='fastparquet')
base_questions = df_base['question'].tolist()

print(f"Loaded {len(base_questions)} base questions")

# Generate hallucination-inducing questions
hallucination_questions = []
target_count = 50

# Sample base questions to work with
sample_size = min(10, len(base_questions))
sampled_questions = random.sample(base_questions, sample_size)

for i, base_question in enumerate(sampled_questions):
    # Generate multiple variations per base question
    questions_per_base = (target_count // sample_size) + (1 if i < (target_count % sample_size) else 0)
    
    prompt = f"""Based on this question: "{base_question}"

Generate {questions_per_base} new questions designed to cause LLM hallucinations (not just incorrect answers, but actual hallucinations involving confabulation of plausible-sounding but false details).

Use these techniques:
- Replace entities with plausible but potentially non-existent names
- Add specific numerical details (dates, percentages, amounts, limits)
- Combine real concepts with fabricated specifics
- Include precise policy/feature details that require exact knowledge
- Use obscure or ambiguous entity names that sound legitimate

Return ONLY the questions, one per line, no numbering or extra text."""

    response = client.messages.create(
        model=os.getenv("ANTHROPIC_MODEL", "claude-sonnet-4-5-20250929"),
        max_tokens=2000,
        messages=[{"role": "user", "content": prompt}]
    )
    
    # Extract questions from response
    generated = response.content[0].text.strip().split('\n')
    generated = [q.strip() for q in generated if q.strip() and not q.strip()[0].isdigit()]
    
    hallucination_questions.extend(generated[:questions_per_base])
    print(f"Generated {len(generated[:questions_per_base])} questions from base question {i+1}/{len(sampled_questions)}")

# Ensure we have exactly 50 questions
hallucination_questions = hallucination_questions[:target_count]

# Create DataFrame and save
df_hallucination = pd.DataFrame({'question': hallucination_questions})
output_file = Path('../data/hallucinationQuestions.parquet')
df_hallucination.to_parquet(output_file, engine='fastparquet', index=False)

print(f"\nTotal hallucination questions generated: {len(hallucination_questions)}")
print(f"Saved to: {output_file}")

In [ ]:
# Test API connection
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("ANTHROPIC_API_KEY")
if not api_key:
    print("ANTHROPIC_API_KEY not found in environment")
elif api_key.startswith("sk-ant-"):
    print(f"API key found: {api_key[:15]}...")
else:
    print("API key found but doesn't match expected format")

# Test network connectivity
import httpx
try:
    response = httpx.get("https://api.anthropic.com", timeout=10.0)
    print(f"Can reach Anthropic API (status: {response.status_code})")
except Exception as e:
    print(f"Cannot reach Anthropic API: {e}")

In [1]:
import pandas as pd
from pathlib import Path

# Load the parquet file
df = pd.read_parquet('../data/hallucinationQuestions_all.parquet')

# Export just the questions column to CSV
df[['question']].to_csv('../data/hallucinationQuestions_all.csv', index=False)

print(f"Exported {len(df)} questions to data/hallucinationQuestions_all.csv")

Exported 13852 questions to data/hallucinationQuestions_all.csv


In [17]:
import pandas as pd
from pathlib import Path
import csv

# Load the parquet file
df = pd.read_parquet('../data/hallucinationDetection_lynx.parquet')

# Export just the questions column to CSV
df = df[df['hallucination_score'] == "FAIL"]
print(df.head())

# Clean questions: remove newlines and ensure proper quoting
df['question'] = df['question'].str.replace('\n', ' ').str.replace('\r', ' ')
df[['question']].to_csv('../data/hallucinationQuestions_lynx.csv', index=False, quoting=csv.QUOTE_ALL)

print(f"Exported {len(df)} questions to data/hallucinationQuestions_lynx.csv")

                                            question  \
0  Can I open an account with Openbank if I resid...   
2  Is it possible to open a joint Openbank accoun...   
3  Does Openbank offer a specific type of account...   
5  Can I apply for an Openbank business account w...   
6  Is there a separate application process for in...   

                                              answer hallucination_score  \
0  OpenBank (also known as Open Bank) is a credit...                FAIL   
2  Yes, it is generally possible to open a joint ...                FAIL   
3  As of my last update in April 2023, there isn'...                FAIL   
5  As of my last update in April 2023, specific p...                FAIL   
6  As of my last update in April 2023, I don't ha...                FAIL   

                                           reasoning  \
0  {"REASONING": ['The CONTEXT does not provide a...   
2  {"REASONING": ['The context does not provide a...   
3  {"REASONING": ['The context does no